In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import random 

from keras.callbacks import History 
history = History()

number_of_samples = 2000
number_of_balanced_dfs = 2

BATCH_SIZE = 10
number_of_attributes = 35
number_of_permutations = 2 # for each attribute

print(tf.__version__)

2.8.2


In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(100, activation='relu'), 
    tf.keras.layers.Dense(11)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
attributes_array = ['10', '15', '17', '22', '24', '29', '31', '32', '33', '34', '36', '37', '38', '45', '46', '47', '48', '59', '60', '61', '62', '63', '64', '79', '80', '81', '82', '83', '84', '87', '88', '89', '90', '91', '92']

In [ ]:
ranking = pd.Series(dtype="float64")
for i in range(number_of_balanced_dfs):
  #Data =processing
  title = '/content/drive/MyDrive/NN-P4/csv-files/df_train_with%d'%number_of_samples + 'samples_per_class%d'%(i+1) + '_11classes.csv'
  df_train = pd.read_csv(title)
  print(title)
  title = '/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_test_with%d'%number_of_samples + 'samples_per_class%d'%(i+1) + '_11classes.csv'
  df_test = pd.read_csv(title)


  target_train = df_train.pop('target')
  target_test = df_test.pop('target')

  df_train = np.asarray(df_train).astype(np.float32)
  target_train = np.asarray(target_train).astype(np.float32)
  df_test_np = np.asarray(df_test).astype(np.float32)
  target_test_np = np.asarray(target_test).astype(np.float32)


  #Train
  # Converts pandas dataframe to tensorflow object
  df_train = tf.convert_to_tensor(df_train)

  # Normalize the data
  normalizer = tf.keras.layers.Normalization(axis=-1)
  normalizer.adapt(df_train)
  #normalizer.adapt(numeric_features_test)

  # Execute Trainning and Testing
  #model = get_basic_model()
  print('Starting training for',number_of_attributes,'attributes')
  model = get_basic_model()
  history = model.fit(df_train, target_train, epochs=8, verbose=2, batch_size=10, callbacks=[history])
  train_accuracy = history.history['accuracy']



  #Test
  print("Test Results")
  test_loss, test_acc =  model.evaluate(df_test_np,  target_test_np, verbose=2 ,batch_size=10)
  tf_predictions_probabilities = model.predict(df_test_np)
  tf_predictions = []

  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)  
    j_max = x.argmax()
    tf_predictions.append(j_max)  

  conf_m = tf.math.confusion_matrix(target_test_np,tf_predictions)
  print(conf_m)

  
  
  #feature importance using permutations
  permutated_accuracy_means_array = []
  #for every attribute
  for attr_i, attr in enumerate(df_test.columns):
    print(f"Running attr: {attr}")  
    
    permutated_accuracy_array = []  
    # run n times for each attribute
    for n in range(number_of_permutations):  
      df_permutated = df_test.copy()
      df_permutated[attr] = np.random.permutation(df_permutated[attr].values)
      #display('original:',numeric_features_test[attr])
      #display('permutaded',df_permutated[attr])

      #df_permutated = tf.convert_to_tensor(df_permutated)      
      permutated_loss, permutated_acc = model.evaluate(df_permutated,  target_test, verbose=0, batch_size=10)  
      permutated_accuracy_array.append(permutated_acc)

    #print('permutated_accuracy_array:',permutated_accuracy_array)
    permutated_accuracy_mean = sum(permutated_accuracy_array) / len(permutated_accuracy_array)  
    #print('permutated_accuracy_mean',permutated_accuracy_mean)
    ratio = permutated_accuracy_mean / test_acc
    #print(f"Original Accuracy: {test_acc:.3f} Permutated Accuracy: {permutated_accuracy_mean:.3f} Ratio: {ratio:.3f}\n")

    permutated_accuracy_means_array.append((permutated_accuracy_mean, attributes_array[attr_i])) 
    #print('permutated_accuracy_means_array',permutated_accuracy_means_array,'\n')
    

  permutated_accuracy_means_array = sorted(permutated_accuracy_means_array)
  print('permutated_accuracy_means_array SORTED',permutated_accuracy_means_array)
  attributes = [x for _, x in permutated_accuracy_means_array]
  print('sorted attributes', attributes)


  for i, attr in enumerate(attributes):
    if attr not in ranking:
        ranking[attr] = 0
    ranking[attr] = ranking[attr] + i



  # Date for report file
  # now = datetime.now()
  # dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
  # Report file content
  # attr_report = {
  #     "datetime": dt_string,
  #     "sorted_attributes_by_weights": attributes
  # }
  # Writes into the file
  # attr_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/%s"%dt_string
  # with open(attr_title, "w") as f:
  #   json.dump(attr_report , f)
  # with open("/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/last-run", "w") as f:
  #   json.dump(attr_report , f)
  # with open(attr_title, "r") as f:
  #   rep = json.load(f)
  #   display(rep)  

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
sorted_ranking = ranking.sort_values()
display(sorted_ranking)
ranking_report = {
    "ranking": sorted_ranking
}
ranking_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/rankings_%s"%dt_string
with open(ranking_title, "w") as f:
  json.dump(ranking_report , f)

/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_train_with2000samples_per_class1_11classes.csv
Starting training for 35 attributes
Epoch 1/7
440/440 - 1s - loss: 1.1930 - accuracy: 0.6365 - 1s/epoch - 3ms/step
Epoch 2/7
440/440 - 1s - loss: 0.6677 - accuracy: 0.7999 - 668ms/epoch - 2ms/step
Epoch 3/7
440/440 - 1s - loss: 0.5359 - accuracy: 0.8327 - 561ms/epoch - 1ms/step
Epoch 4/7
440/440 - 1s - loss: 0.4619 - accuracy: 0.8516 - 557ms/epoch - 1ms/step
Epoch 5/7
440/440 - 1s - loss: 0.4116 - accuracy: 0.8690 - 546ms/epoch - 1ms/step
Epoch 6/7
440/440 - 1s - loss: 0.3733 - accuracy: 0.8791 - 549ms/epoch - 1ms/step
Epoch 7/7
440/440 - 1s - loss: 0.3429 - accuracy: 0.8910 - 543ms/epoch - 1ms/step
Test Results
22000/22000 - 23s - loss: 0.6421 - accuracy: 0.8733 - 23s/epoch - 1ms/step
tf.Tensor(
[[1658    2    7   16  121  176    2    5    1    6    6]
 [   0 1754   60   26   10   38   24    3   40   37    8]
 [   0   17 1879   28    4    5   48    0    0    0   19]
 

KeyboardInterrupt: ignored

In [ ]:
# 10 nodos
# ['83', '82', '81', '22', '15', '90', '79', '89', '84', '60', '88', '87', '29', '80', '63', '62', '17', '10', '59', '64', '24', '61', '48', '36', '31', '47', '33', '32', '34', '45', '91', '37', '38', '46', '92']
# ['83', '82', '81', '15', '22', '88', '90', '79', '89', '84', '60', '87', '10', '29', '64', '80', '17', '63', '59', '62', '24', '61', '45', '91', '31', '32', '47', '38', '34', '36', '33', '37', '46', '48', '92']
# ['83', '82', '81', '15', '22', '79', '90', '84', '88', '10', '60', '89', '29', '87', '80', '17', '64', '63', '62', '59', '24', '91', '61', '33', '34', '31', '48', '45', '37', '32', '36', '47', '38', '46', '92']
# ['83', '82', '81', '15', '22', '84', '89', '79', '90', '17', '10', '87', '64', '80', '88', '29', '59', '60', '63', '24', '91', '61', '62', '32', '46', '38', '34', '33', '37', '36', '48', '47', '31', '45', '92']
# ['83', '82', '81', '90', '88', '79', '84', '89', '15', '22', '29', '10', '64', '17', '62', '59', '80', '87', '63', '61', '60', '24', '91', '48', '45', '33', '36', '34', '46', '37', '32', '47', '31', '38', '92']

# 100 nodos
# ranking = pd.Series(dtype="float64")
# importance_order = ['82', '83', '81', '89', '22', '15', '90', '84', '87', '88', '79', '17', '64', '80', '10', '59', '29', '60', '24', '63', '61', '91', '62', '48', '32', '34', '47', '31', '45', '33', '46', '38', '36', '37', '92']
# for i, attr in enumerate(importance_order):
#     if attr not in ranking:
#         ranking[attr] = 0
#     ranking[attr] = ranking[attr] + i

# importance_order = ['82', '83', '81', '89', '15', '22', '84', '88', '90', '79', '87', '60', '80', '64', '10', '17', '29', '63', '59', '24', '62', '61', '91', '34', '47', '32', '31', '33', '48', '37', '45', '36', '46', '92', '38']
# for i, attr in enumerate(importance_order):
#     if attr not in ranking:
#         ranking[attr] = 0
#     ranking[attr] = ranking[attr] + i

# importance_order = ['82', '83', '81', '89', '15', '84', '22', '90', '88', '79', '87', '80', '64', '29', '17', '10', '63', '62', '59', '61', '24', '60', '91', '48', '32', '31', '34', '33', '47', '37', '46', '45', '36', '38', '92']
# for i, attr in enumerate(importance_order):
#     if attr not in ranking:
#         ranking[attr] = 0
#     ranking[attr] = ranking[attr] + i

# importance_order = ['83', '82', '81', '89', '84', '15', '22', '90', '88', '87', '79', '80', '60', '17', '29', '64', '10', '63', '59', '24', '61', '62', '48', '32', '34', '31', '47', '46', '33', '45', '36', '91', '38', '37', '92']
# for i, attr in enumerate(importance_order):
#     if attr not in ranking:
#         ranking[attr] = 0
#     ranking[attr] = ranking[attr] + i

# importance_order = ['82', '83', '81', '89', '84', '22', '15', '88', '79', '90', '87', '60', '80', '59', '17', '10', '29', '64', '63', '62', '24', '61', '47', '91', '48', '31', '33', '36', '45', '34', '32', '37', '46', '38', '92']
# for i, attr in enumerate(importance_order):
#     if attr not in ranking:
#         ranking[attr] = 0
#     ranking[attr] = ranking[attr] + i

# sorted_ranking = ranking.sort_values()
# display(sorted_ranking)
# sorted_ranking_index = sorted_ranking.index
# display(sorted_ranking_index)
# ranking_report = {
#     "ranking": ['82', '83', '81', '89', '15', '22', '84', '90', '88', '79', '87', '80', '17', '64', '60', '10', '29', '59', '63', '24', '62', '61', '91', '48', '47', '32', '34', '31', '33', '45', '46', '36', '37', '38', '92']
# }
# ranking_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/rankings_%s"%current_date_and_time_string
# with open(ranking_title, "w") as f:
#   json.dump(ranking_report , f)